In [1]:
import os
import requests
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions

options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)
driver.get('https://www.screener.in')

input("Press Enter after you have searched for a company and the desired company page has loaded")

soup = BeautifulSoup(driver.page_source, 'html.parser')

#Storing Company Name
name_section = driver.find_element_by_xpath('/html/body/div/div[1]/h1')
company = name_section.text

#Storing Company Description (About Section)
about_section = driver.find_element_by_xpath('//*[@id="top"]/div[3]/div[1]/div[1]/div[2]/p')
about_text = about_section.text
about_text = about_text[:-3]

#Storing Company Details Section
data_section = soup.find('ul', {'id': 'top-ratios'})
data_items = data_section.find_all('li')
data_dict = {}
for item in data_items:
    key = item.find('span', {'class': 'name'}).text.strip()
    value = ' '.join(item.find('span', {'class': 'value'}).text.split())
    data_dict[key] = value
df_data_dict = pd.DataFrame(data_dict, index=[1])

#Storing Company's Quarterly Results Section
table = soup.find('section', {'id': 'quarters'}).find('table')
data = []
headers = [header.text.strip() for header in table.find_all('th')]
rows = table.find_all('tr')
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
df_quarters = pd.DataFrame(data, columns=headers)
df_quarters = df_quarters[:-1]
df_quarters = df_quarters.set_index("")

#Storing Company's Profit & Loss Section
table = soup.find('section', {'id': 'profit-loss'}).find('table')
data = []
headers = [header.text.strip() for header in table.find_all('th')]
rows = table.find_all('tr')
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
df_profit_loss = pd.DataFrame(data, columns=headers)
df_profit_loss = df_profit_loss.set_index("")

#Storing Company's Balance Sheet Section
table = soup.find('section', {'id': 'balance-sheet'}).find('table')
data = []
headers = [header.text.strip() for header in table.find_all('th')]
rows = table.find_all('tr')
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() if ele.text.strip() else np.nan for ele in cols]
    data.append([ele for ele in cols if ele])
df_balance_sheet = pd.DataFrame(data, columns=headers)
df_balance_sheet = df_balance_sheet.set_index("")

#Storing Company's Cash Flow Section
table = soup.find('section', {'id': 'cash-flow'}).find('table')
data = []
headers = [header.text.strip() for header in table.find_all('th')]
rows = table.find_all('tr')
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
df_cash_flow = pd.DataFrame(data, columns=headers)
df_cash_flow = df_cash_flow.set_index("")

#Storing Company's Ratios Section
table = soup.find('section', {'id': 'ratios'}).find('table')
data = []
headers = [header.text.strip() for header in table.find_all('th')]
rows = table.find_all('tr')
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() if ele.text.strip() else np.nan for ele in cols]
    data.append([ele for ele in cols if ele])
df_ratios = pd.DataFrame(data, columns=headers)
df_ratios = df_ratios.set_index("")

#Storing Company's Shareholding Section
table = soup.find('section', {'id': 'shareholding'}).find('table')
data = []
headers = [header.text.strip() for header in table.find_all('th')]
rows = table.find_all('tr')
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
df_shareholding = pd.DataFrame(data, columns=headers)
df_shareholding = df_shareholding.set_index("")

driver.quit()

'''---------------------------------------------------------------------------------
    Storing Top 20 Articles about the searched company from Money Control Website
---------------------------------------------------------------------------------'''

try:
    
    def get_top_moneycontrol_articles(query, num_articles=20):
        url = f"https://www.moneycontrol.com/news/tags/{query}.html"
        req = requests.get(url)
        soup = BeautifulSoup(req.content, "html.parser")
        articles = soup.find_all("li", class_="clearfix")
        top_articles = []
        for i, article in enumerate(articles[:num_articles], start=1):
            title = article.find("h2").text.strip()
            link = article.find("a")["href"]
            top_articles.append({"Article No": i, "Title": title, "Link": link})
        return pd.DataFrame(top_articles)

    # Obtain rop 20 articles of the company
    phrase = company.strip(' Ltd')
    top_articles = get_top_moneycontrol_articles(phrase, num_articles=20)
    top_articles = top_articles.set_index("Article No")
    
except Exception as e:
    print("Articles not available")

'''---------------------------------------------------------------------------------
                            Converting into excel report
---------------------------------------------------------------------------------'''

df1 = df_data_dict.copy()
df2 = df_quarters.copy()
df3 = df_profit_loss.copy()
df4 = df_balance_sheet.copy()
df5 = df_cash_flow.copy()
df6 = df_ratios.copy().copy()
df7 = df_shareholding.copy()
df8 = top_articles.copy() if 'top_articles' in locals() else None

# Define the titles for each DataFrame
titles = ['Data Dictionary', 'Quarterly Results', 'Profit & Loss', 'Balance Sheet', 
          'Cash Flows', 'Ratios', 'Shareholding Pattern','Top Articles']

directory = "Screener Reports"
if not os.path.exists(directory):
    os.makedirs(directory)
    
# Create a Pandas Excel writer using XlsxWriter as the engine
file_name = "Screener Reports/" + company + ".xlsx"
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
workbook = writer.book

# Define a format for the head title cell
title_format_head = workbook.add_format({'bold': True, 'font_size': 24, 'align': 'center', 'underline' : True})

# Define a format for the title cells
title_format = workbook.add_format({'bold': True, 'font_size': 18, 'align': 'center'})

# Create a format for bold text
bold_border = workbook.add_format({'bold': True, 'border': 1, 'font_size': 14})

# Create a format for borders
border = workbook.add_format({'border': 1, 'font_size': 14})

# Create a format for font size 14
font_size_14 = workbook.add_format({'font_size': 14})

# Create a format for font size 14 bold
font_size_14_bold = workbook.add_format({'font_size': 14, 'bold' : True})

# Create a format for font size 14 not bold
font_size_14_notbold = workbook.add_format({'font_size': 14, 'bold' : False})

# Write the company name as the title of the Excel sheet
worksheet = workbook.add_worksheet('Sheet1')
worksheet.write('G1', company, title_format_head)

# Write the "About" section title
worksheet.write('G4', 'About', title_format)

# Write the "About" section text
worksheet.write('A6', about_text, font_size_14)

# Write each DataFrame to the sheet with appropriate spacing and titles
row = 8  # Start row for the first DataFrame

# Write the title for the first DataFrame
worksheet.merge_range(row, 1, row, df1.shape[1]+3, 'Company Details', title_format)
row += 2  # Leave a blank row after the title

# Write the first DataFrame without index and with an empty cell between each column
num_columns = df1.shape[1]
start_col = 1
for col in range(num_columns):
    worksheet.write(row, start_col, df1.columns[col], bold_border)
    start_col += 2
row += 1  # Move to the next row for data

# Write the data in the first DataFrame with an empty cell between each column
for col in range(num_columns):
    values = df1.iloc[:, col].tolist()
    worksheet.write_column(row, col*2+1, values, border)
    
row += len(df1) + 3  # Leave a blank row after the DataFrame

for i, (title, df) in enumerate(zip(titles[1:], [df2, df3, df4, df5, df6, df7, df8]), start=2):
    
    if df is not None:
        
        # Convert the data in the DataFrame to numeric type
        for col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='ignore')

        # Replace NaN or INF values with empty strings
        df = df.replace([np.inf, -np.inf], np.nan).fillna('')

        # Write the title for the DataFrame
        worksheet.write(row, 6, title, title_format)  # Column F (index 5)
        row += 2  # Leave a blank row after the title

        # Write index manually with font size 16
        worksheet.write_column(row + 1 ,1 ,df.index.tolist(), font_size_14_bold)

        # Write columns manually with font size 16
        for j,col in enumerate(df.columns):
            worksheet.write(row ,j + 2 ,col , font_size_14_bold)

        # Write data of dataframe manually with font size 16
        for r in range(len(df)):
            worksheet.write_row(row + r + 1 ,2 ,df.iloc[r,:].tolist(), font_size_14_notbold)

        row += len(df) + 3 
    
# Set column width to make it wider and more readable 
worksheet.set_column(0 , 100 , 12)
worksheet.set_column('B:B', 22)

# Save the Excel file
writer.close()

Press Enter after you have searched for a company and the desired company page has loaded
